In [ ]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
#from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def calculate_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss.replace(0, 1e-6)
    return 100 - (100 / (1 + rs))

def feature_engineering(df):
    import numpy as np
    """
    Takes a preprocessed DataFrame and creates all necessary time-series features.
    """
    print("--- Performing Feature Engineering ---")
    if len(df) < 15:
        error_msg = f'Dataframe size is less than 15 ({len(df)} rows found). Cannot create features. Get more data.'
        print(f"🚨 {error_msg}")
        raise ValueError(error_msg) # Stop execution here
        
    df_feat = df.copy()
    df_feat['rsi_14'] = calculate_rsi(df_feat, window=14)
    df_feat['lag_1'] = df_feat['close'].shift(1)
    df_feat['lag_5'] = df_feat['close'].shift(5)
    df_feat['lag_10'] = df_feat['close'].shift(10)
    df_feat['rolling_mean_5'] = df_feat['close'].rolling(window=5).mean()
    df_feat['rolling_std_5'] = df_feat['close'].rolling(window=5).std()
    df_feat['rolling_mean_10'] = df_feat['close'].rolling(window=10).mean()
    df_feat['rolling_std_10'] = df_feat['close'].rolling(window=10).std()
    log_return = np.log(df_feat['close'] / df_feat['close'].shift(1))
    df_feat['volatility_7'] = log_return.rolling(window=7).std() # 7-day volatility
    df_feat['momentum_5'] = log_return.rolling(window=5).mean() # 5-day momentum (average return)
    df_feat['rsi_14'] = calculate_rsi(df_feat, window=14)
    df_feat['day_of_week'] = df_feat['merge_date'].dt.dayofweek
    df_feat['high_low_spread'] = df_feat['high'] - df_feat['low']
    df_feat['momentum_x_volume'] = df_feat['momentum_5'] * df_feat['volume']
    df_feat['rsi_sq'] = df_feat['rsi_14']**2
    
    return df_feat

df = pd.read_csv(r'/kaggle/input/bitcoin-and-us-treasury-with-daily-sentiment/bitcoin_sentiment_12012022_11082025.csv')
df['target'] = df['close'].shift(-1)
df['merge_date'] = pd.to_datetime(df['merge_date'], errors='coerce')
df = feature_engineering(df)

# --- 1. Define Features ---
# The 'close' column is now a feature, not the target.
# We must exclude the new 'target' column from the features.
features = [col for col in df.columns if col not in ['merge_date', 'datetime_utc', 'timestamp', 'target'] and 'unnamed' not in col]

# --- 2. Clean Data ---
# Define the columns that MUST NOT have NaNs for training
required_cols = [col for col in df.columns if col not in ['merge_date', 'datetime_utc', 'timestamp', 'unnamed'] and 'unnamed' not in col]
required_cols.append('target') # Ensure the new target is included

# Drop rows based on the required columns
data_clean = df.dropna(subset=required_cols).reset_index(drop=True)

# Check the result after cleaning
if data_clean.empty:
    raise ValueError("The dataset is empty after dropping NaNs. Check your feature engineering steps for excessive NaN creation!")

X = data_clean[features]
y = data_clean['target'] # Use the new 'target' column

# --- 3. Split and Train ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


selected_features = ['open', 'high', 'low', 'close', 'volume', 'weighted_sentiment', 'lag_1', 'lag_10', 
                     'rolling_mean_5', 'rolling_mean_10', 'rolling_std_10', 'volatility_7', 'momentum_5', 'high_low_spread', 'momentum_x_volume', 'rsi_sq']
X_train = X_train[selected_features]   
X_test = X_test[selected_features]
features = selected_features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
########################

print("--- Training and Evaluating Models ---")
results = {}


# 4. Train ML Models
mlr_model = LinearRegression().fit(X_train_scaled, y_train)
ridge_model = Ridge(alpha=0.5, random_state=42).fit(X_train_scaled, y_train)
rf_model = RandomForestRegressor(n_estimators=600, random_state=42).fit(X_train_scaled, y_train)
xgb_model = xgb.XGBRegressor(colsample_bytree=1.0, learning_rate=0.03, max_depth= 7,n_estimators=1200, subsample = 0.9,
                             objective='reg:squarederror', random_state=42).fit(X_train_scaled, y_train)

# Train Auto-ARIMA on the training data for a fair comparison
#arima_model = ARIMA(y_train, order=(1,1,3))
#arima_model_fit = arima_model.fit()

# Evaluate Models
mlr_preds = mlr_model.predict(X_test_scaled)
ridge_preds = ridge_model.predict(X_test_scaled)
rf_preds = rf_model.predict(X_test_scaled)
xgb_preds = xgb_model.predict(X_test_scaled)
#arima_preds = arima_model_fit.forecast(steps=len(y_test))

results['Linear Regression'] = {'MAE': mean_absolute_error(y_test, mlr_preds)}
results['Ridge Regression (0.5)'] = {'MAE': mean_absolute_error(y_test, ridge_preds)}
results['Random Forest'] = {'MAE': mean_absolute_error(y_test, rf_preds)}
results['XGBoost'] = {'MAE': mean_absolute_error(y_test, xgb_preds)}

#dropped ARIMA model. Models with more context are outperforming ARIMA. 
#results[f'ARIMA (1,1,3)'] = {'MAE': mean_absolute_error(y_test, arima_preds)}

results_df = pd.DataFrame(results).T
print("\n--- Model Performance on Test Set ---")
print(results_df.sort_values(by='MAE'))